Взять метод обучения с подкреплением, в котором есть применение нейросети и применить подбор гипер параметров

In [ ]:
!pip install gym numpy torch optuna

In [6]:
import gym

# Создаем среду
env = gym.make("FrozenLake-v1") # https://github.com/openai/

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
import optuna

# Определение нейронной сети для аппроксимации Q-функции
class QNetwork(nn.Module):
    def __init__(self, state_size, action_size, hidden_size=24):
        super(QNetwork, self).__init__()
        self.fc1 = nn.Linear(state_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, action_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Среда (упрощенная для примера)
class SimpleEnv:
    def __init__(self):
        self.state_space_size = 4
        self.action_space_size = 2

    def step(self, state, action):
        next_state = (state + 1) % self.state_space_size
        reward = 1 if action == 0 else -1
        done = next_state == 0
        return next_state, reward, done

    def reset(self):
        return 0

def train(env, model, episodes, learning_rate, gamma):
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    loss_fn = nn.MSELoss()
    total_reward = 0

    for episode in range(episodes):
        state = env.reset()
        done = False
        episode_reward = 0

        while not done:
            state_tensor = torch.FloatTensor([state])
            q_values = model(state_tensor)

            _, action = torch.max(q_values, dim=0)
            action = action.item()

            next_state, reward, done = env.step(state, action)
            episode_reward += reward

            next_state_tensor = torch.FloatTensor([next_state])
            next_q_values = model(next_state_tensor)

            target_q = reward + gamma * torch.max(next_q_values)
            loss = loss_fn(q_values[action], target_q)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            state = next_state
        
        total_reward += episode_reward
    
    average_reward = total_reward / episodes
    print(average_reward)
    return average_reward

def objective(trial):
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
    gamma = trial.suggest_uniform('gamma', 0.85, 0.99)
    hidden_size = trial.suggest_categorical('hidden_size', [16, 32, 64, 128])
    episodes = 100

    model = QNetwork(1, 2, hidden_size)
    env = SimpleEnv()
    average_reward = train(env, model, episodes, learning_rate, gamma)

    return average_reward

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

print("Лучшие гиперпараметры:", study.best_params)

[I 2024-03-07 17:26:16,502] A new study created in memory with name: no-name-ef8e6b19-a178-4ac1-94d1-4b247bcc6dca


/var/folders/9h/5fdk26117gjcqkjk01yhxr5w0000gn/T/ipykernel_27064/1178418317.py:74: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
/var/folders/9h/5fdk26117gjcqkjk01yhxr5w0000gn/T/ipykernel_27064/1178418317.py:75: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  gamma = trial.suggest_uniform('gamma', 0.85, 0.99)
[I 2024-03-07 17:26:16,668] Trial 0 finished with value: 3.22 and parameters: {'learning_rate': 0.0022232378052516634, 'gamma': 0.9245754534211699, 'hidden_size': 128}. Best is trial 0 with value: 3.22.
[I 2024-03-07 17:26:16,804] Trial 1 finished with value: 4.0 and parameters: {'learning_rate': 0.06980575140348894

3.22
4.0


[I 2024-03-07 17:26:16,932] Trial 2 finished with value: -4.0 and parameters: {'learning_rate': 3.7736906667717635e-05, 'gamma': 0.8896824130573853, 'hidden_size': 32}. Best is trial 1 with value: 4.0.
[I 2024-03-07 17:26:17,060] Trial 3 finished with value: 4.0 and parameters: {'learning_rate': 0.09730412550444388, 'gamma': 0.8520739144131977, 'hidden_size': 32}. Best is trial 1 with value: 4.0.


-4.0
4.0


...

3.92
4.0


[I 2024-03-07 17:26:29,688] Trial 96 finished with value: 4.0 and parameters: {'learning_rate': 0.007625332207712092, 'gamma': 0.8607408792093729, 'hidden_size': 128}. Best is trial 1 with value: 4.0.
[I 2024-03-07 17:26:29,838] Trial 97 finished with value: 3.98 and parameters: {'learning_rate': 0.037026074729780034, 'gamma': 0.9727021100197579, 'hidden_size': 32}. Best is trial 1 with value: 4.0.


4.0
3.98


[I 2024-03-07 17:26:29,971] Trial 98 finished with value: 3.98 and parameters: {'learning_rate': 0.08765066381768837, 'gamma': 0.9275376092111716, 'hidden_size': 128}. Best is trial 1 with value: 4.0.
[I 2024-03-07 17:26:30,107] Trial 99 finished with value: 4.0 and parameters: {'learning_rate': 1.297657258073961e-05, 'gamma': 0.8909264848520577, 'hidden_size': 16}. Best is trial 1 with value: 4.0.


3.98
4.0
Лучшие гиперпараметры: {'learning_rate': 0.06980575140348894, 'gamma': 0.8657160315938123, 'hidden_size': 128}


Функция 'objective' определяет объектив для оптимизации с помощью библиотеки Optuna. Optuna используется для автоматического выбора оптимальных гиперпараметров.
В objective, гиперпараметры (скорость обучения, gamma, размер скрытого слоя) варьируются в предопределенных диапазонах. Модель обучается с этими параметрами и возвращается средняя награда как метрика качества.
Наконец, создается исследование (study), которое максимизирует среднюю награду и Optuna ищет лучшие гиперпараметры, выполняя функцию objective множество раз
